# Big Data para Dados Públicos

##### Maciel C. Vidal

Na aula de hoje iremos conferir como criar, em um Jupyter Notebook, uma conexão com um SGBD para recuperar dados. Os dados desejados serão especificados utilizando SQL.


## Instalando as bibliotecas necessárias

Para criar uma conexão com o PostgreSQL, iremos utilizar a biblioteca `psycopg2`. Tente realizar o import dela e se não for possível, descomente e execute a seguinte linha para instalação:

In [ ]:
# !python -m pip install psycopg2-binary

   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   ---------------------------------------- 1.2/1.2 MB 19.2 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Para criação de gráficos interativos, vamos utilizar a biblioteca `plotly.express`. Tente realizar o import dela e se não for possível, descomente e execute a seguinte linha para instalação:

In [ ]:
# !python -m pip install plotly_express


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## Importando as bibliotecas necessárias

Agora, vamos importar as bibliotecas necessárias:

In [3]:
# Para utilizar recursos do sistema
import os
import sys

# Para plot de gráficos
%matplotlib inline
import matplotlib.pyplot as plt
import plotly.express as px

# Para solicitar senha
import getpass

# Dataframe: tratar dados no Python
import pandas as pd

# Para conectar ao SGBD
import psycopg2 as pg


Caso obtenha algum erro, utilize o **!pip install** para instalar a biblioteca ausente!


Você também pode conferir de onde está executando o Python e qual a versão

In [4]:
print("Executável:")
print(sys.executable)

print("\nVersão do Python:")
print(sys.version)

Executável:
c:\Users\guoli\Documents\Insper\BigData\venv\Scripts\python.exe

Versão do Python:
3.12.7 (tags/v3.12.7:0b05ead, Oct  1 2024, 03:06:41) [MSC v.1941 64 bit (AMD64)]


Vamos conferir em qual diretório iremos trabalhar (é o diretório do notebook)

In [5]:
print("O seu notebook está na pasta:")
print(os.getcwd())

O seu notebook está na pasta:
c:\Users\guoli\Documents\Insper\BigData\Aula_12


## Conectando à base de dados

O nosso objetivo final aqui é realizar alguma análise de dados (ex: extrair valores vendidos no mês, criar gráfico com quantidade de vendas). Como os dados estão armazenados em um SGBD, precisaremos criar uma conexão para permitir esta troca de informações:

<img src="https://bigdata-22-2.s3.us-east-2.amazonaws.com/sql/pandas_sql.png">

Para isso, primeiro iremos configurar algumas variáveis que irão conter as informações da conexão (caminho até o servidor, porta, database, usuário, senha):

In [6]:
host = "18.117.69.137"
port = 5432
db = "aulas"
username = "gustavoeso"
# Para encontrar a senha: procure, no seu e-mail, por "feedbackinsper"
# password = ""
# Se não quiser deixar a senha escrita no notebook,
# descomente a próxima linha. Será solicitado que informe a senha a cada execução.
password = getpass.getpass("Digite a senha: ")

### Um primeiro exemplo de conexão

Para criar uma conexão, utilizaremos `pg.connect`, passando as informações necessárias (caminho até o servidor, porta, database, usuário, senha)

In [7]:
conn = pg.connect(host=host, port=port, database=db, user=username, password=password)

cur = conn.cursor()

Agora a conexão existe e podemos utilizá-la para recuperar informações, por exemplo, sobre nossos clientes

In [8]:
query = "SELECT * FROM olist.customer LIMIT 2"

cur.execute(query)

dados_clientes = cur.fetchall()

print(dados_clientes)

[('06b8999e2fba1a1fbc88172c00ba8bc7', '861eff4711a542e4b93843c6dd7febb0', 14409, 'franca', 'SP'), ('18955e83d337fd6b2def6b18a428ac77', '290c77bc529b7ac935b93aa66c333dc3', 9790, 'sao bernardo do campo', 'SP')]


### Fechando a conexão

Quando a conexão não for mais necessária, é indicado que a conexão e o cursor sejam fechados, liberando recurso para os demais usuários.

In [9]:
cur.close()
conn.close()

## Base de dados pública da Olist

Nesta aula, utilizaremos uma base pública, a **Brazilian E-Commerce Public Dataset by Olist**. Ela está disponível no **Kaggle** (https://www.kaggle.com/olistbr/brazilian-ecommerce), uma fonte legal para conseguir dados públicos e evoluir seus conhecimentos em Machine Learning.

<img src="https://bigdata-22-2.s3.us-east-2.amazonaws.com/sql/olist_db.png">

Os dados da base já foram baixados pelo professor e inseridos no SGBD PostgreSQL no **schema** **olist**. A relação entre as tabelas é:

<img src="https://bigdata-22-2.s3.us-east-2.amazonaws.com/sql/olist_der.png">

## Pandas

Para facilitar nosso trabalho com os dados tabulares extraídos do SGBD, iremos utilizar as bibliotecas Pandas e Dask para armazenar os dados em DataFrames. Uma outra facilidade é que o Pandas e Dask já possuem uma API para recuperar dados direto de SGBDs utilizando SQL

Vamos criar uma função que cria uma conexão, recupera dados e retorna um DataFrame:

In [10]:
def get_pandas_df(sql):
    conn = pg.connect(host=host, port=port, database=db, user=username, password=password)
    
    df = pd.read_sql_query(sql, conn)
    
    conn.close()
    
    return df

Pronto! Agora podemos executar qualquer query e ter os resultados disponíveis!

In [11]:
sql = """
SELECT id, city as cidade, state as uf
FROM olist.customer c
LIMIT 5
"""

df_cliente = get_pandas_df(sql)

C:\Users\guoli\AppData\Local\Temp\ipykernel_11136\456190472.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(sql, conn)


Exibindo os resultados!

In [12]:
dict(df_cliente.dtypes)

{'id': dtype('O'), 'cidade': dtype('O'), 'uf': dtype('O')}

In [13]:
df_cliente

,id,cidade,uf
0,06b8999e2fba1a1fbc88172c00ba8bc7,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,sao paulo,SP
3,b2b6027bc5c5109e529d4dc6358b12c3,mogi das cruzes,SP
4,4f2d8ab171c80ec8364f7c12e35b23ad,campinas,SP


## Pandas + SQLAlchemy

Apesar de funcionar, nas células anteriores você pode ter recebido um aviso que Pandas oficialmente suporta apenas conexões pela biblioteca **SQLAlchemy**. Vamos verificar como ficaria com esta biblioteca!

Caso necessário, descomente a próxima linha e faça a instalação

In [ ]:
# !python -m pip install sqlalchemy

   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ---------------------------------------- 2.1/2.1 MB 14.6 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Vamos importar a biblioteca

In [15]:
import sqlalchemy

Verifique as alterações na função `get_pandas_df` para funcionar com sqlalchemy:

In [16]:
def get_pandas_df(sql):
    engine = sqlalchemy.create_engine(f"postgresql+psycopg2://{username}:{password}@{host}:{port}/{db}")

    df = pd.read_sql_query(sql, engine)

    engine.dispose()
    return df

E testar o mesmo exemplo utilizado anteriormente

In [17]:
sql = """
SELECT *
  FROM olist.customer
 LIMIT 5
"""

df_cliente = get_pandas_df(sql)

df_cliente.head()

,id,unique_id,zip_code_prefix,city,state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP


## Pandas: Lendo Tabela Completa

Com o uso da função `pd.read_sql_query` conseguimos executar qualquer query. Isto permite realizar *JOIN* entre tabelas, utilizar funções de agrupamento e agregação, etc. Entretanto, em certas situações já temos uma tabela ou *VIEW* com os dados disponíveis. Nestas situações, o Pandas disponibiliza a função `read_sql_table`.

Vamos ver um exemplo:

In [18]:
engine = sqlalchemy.create_engine(f"postgresql+psycopg2://{username}:{password}@{host}:{port}/{db}")

df_clientes = pd.read_sql_table("customer", engine, schema="olist")

engine.dispose()

df_clientes.head()

,id,unique_id,zip_code_prefix,city,state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP


In [19]:
def get_pandas_table(table, schema):
    engine = sqlalchemy.create_engine(f"postgresql+psycopg2://{username}:{password}@{host}:{port}/{db}")
    df = pd.read_sql_table(table, engine, schema=schema)
    engine.dispose()
    return df

## Processando os dados: API DataFrame

Para processar dados provenientes de algum SGBD como o PostgreSQL, uma opção é ler os dados brutos de uma tabela e processar com a API de DataFrames do Pandas.

In [20]:
df_cliente = get_pandas_table("customer", "olist")

Uma amostra dos dados:

In [21]:
df_cliente.head(2)

,id,unique_id,zip_code_prefix,city,state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP


Quantidade de linhas e colunas

In [22]:
df_cliente.shape

(99441, 5)

Vamos realizar a contagem do número de clientes por cidade

In [23]:
df_cliente["city"].value_counts(True).round(3).to_frame().head(6)

,proportion
city,
sao paulo,0.156
rio de janeiro,0.069
belo horizonte,0.028
brasilia,0.021
curitiba,0.015
campinas,0.015


## Processando os dados com SQL

Uma outra opção é processar os dados utilizando uma query e recuperar apenas os resultados:

In [24]:
# Preencha aqui
sql = """
SELECT c.city,
       ROUND(COUNT(c.city) /
               (SELECT count(1)::NUMERIC
                FROM olist.customer), 3) AS proportion
FROM olist.customer AS c
GROUP BY c.city
ORDER BY proportion DESC
LIMIT 6
"""

df_city_dist = get_pandas_df(sql)
df_city_dist

,city,proportion
0,sao paulo,0.156
1,rio de janeiro,0.069
2,belo horizonte,0.028
3,brasilia,0.021
4,campinas,0.015
5,curitiba,0.015


## Gerando gráficos com Plotly Express

Vamos utilizar o `plotly.express` para gerar gráficos interativos. Veja mais em https://plotly.com/python/plotly-express/

In [25]:
sql = """
SELECT c.state AS UF,
       ROUND(COUNT(c.state) /
               (SELECT count(1)::NUMERIC
                FROM olist.customer), 3) AS freq_clientes
FROM olist.customer AS c
GROUP BY c.state
ORDER BY freq_clientes DESC
LIMIT 6
"""

df_uf_dist = get_pandas_df(sql)
df_uf_dist

,uf,freq_clientes
0,SP,0.420
1,RJ,0.129
2,MG,0.117
3,RS,0.055
4,PR,0.051
5,SC,0.037


In [26]:
px.bar(df_uf_dist, x="uf", y="freq_clientes")

Vamos atualizar informações do gráfico como labels dos eixos para melhorar a exibição!

In [27]:
# Alguns templates
# ["plotly", "plotly_white", "plotly_dark", "ggplot2", "seaborn", "simple_white", "none"]

px.bar(df_uf_dist, x="uf", y="freq_clientes",
       labels={"freq_clientes": "Frequência", "uf": "Estado"},
       title="Distribuição dos clientes por UF", color="freq_clientes",
       template="ggplot2")

## Plot de Mapas com Folium

Vamos utilizar o `folium` para gerar gráficos interativos com informações georeferenciadas. Veja mais em https://python-visualization.github.io/folium/ e https://www.openstreetmap.org

Importe a biblioteca folium. Caso não seja possível, realize a instalação descomentando a seguinte célula:

In [ ]:
# !python -m pip install folium


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Vamos importar as bibliotecas necessárias

In [29]:
# Para mapas
import folium

# Para mapa de calor
from folium.plugins import HeatMap

Vamos exibir um primeiro mapa centrado na cidade de são paulo

In [30]:
mapa = folium.Map(
    location=[-23.5489, -46.6388],
    zoom_start=12
)

mapa

Agora, vamos recuperar informações de alguns clientes (latitude e longitute) e plotar no mapa

In [31]:
# recupera aleatoriamente 3% da base

sql = """
SELECT g.lat,
       g.lng
FROM olist.seller s,
     olist.geolocation g
WHERE s.zip_code_prefix = g.zip_code_prefix
  AND random() < 0.03
"""

Recuperando os dados

In [32]:
df_heat = get_pandas_df(sql)
df_heat.head()

,lat,lng
0,-22.979130,-47.000473
1,-21.994650,-47.856876
2,-22.003489,-47.853900
3,-22.003550,-47.856823
4,-21.747100,-48.831432


In [33]:
df_heat.shape

(13111, 2)

Transformando em listas

In [34]:
lat_list = df_heat["lat"].to_list()
lng_list = df_heat["lng"].to_list()
print(lat_list)

[-22.97913, -21.99465, -22.003489, -22.00355, -21.7471, -21.75836, -21.759016, -21.753855, -21.757717, -21.760866, -21.74872, -21.768026, -21.750257, -21.746813, -21.748966, -21.772173, -21.774868, -21.753506, -21.757952, -21.762287, -21.754925, -21.75086, -21.751472, -21.764753, -21.281208, -21.30318, -21.289543, -21.296738, -21.29878, -21.295654, -21.291655, -21.308088, -21.749937, -21.746813, -21.747807, -21.751348, -21.768127, -21.750795, -21.753874, -21.756401, -21.756306, -21.768026, -21.750114, -23.572027, -23.571762, -23.5748, -23.572647, -21.751307, -21.759838, -21.756739, -21.754047, -21.745007, -21.75861, -21.748545, -21.763147, -21.738056, -21.734663, -22.926819, -22.93013, -22.921307, -22.918955, -22.929544, -22.933134, -22.925316, -22.925077, -22.926104, -22.928757, -22.919273, -22.923159, -21.186642, -21.21044, -21.199694, -21.188606, -21.19101, -21.185534, -21.210104, -21.207245, -21.206089, -21.210104, -21.204584, -21.204693, -21.205822, -23.539257, -23.536894, -23.531

Exibindo o mapa de calor

In [35]:
mapa = folium.Map(
    location=[-23.5489, -46.6388],
    zoom_start=4
)

HeatMap(list(zip(lat_list, lng_list))).add_to(mapa)

mapa

## Exercício 1

Identifique, na base olist.order_item, quais são os 20 produtos com mais vendas (valor total). Exiba um DataFrame contendo as informações do product_id e valor total faturado

In [ ]:
sql = """
 -- Sua query aqui
 """

df_top20 = get_pandas_df(sql)
df_top20

## Exercício 2

Exiba as informações do Exercício 1 em um gráfico de barras

## Exercício 3

Na base de ordens (`olist.order`), quais são os possíveis status para as compras?

## Exercício 4

Realize uma alteração na query do Exercício 1 para considerar apenas ordens com status entregue. Exiba um DataFrame e também um gráfico de barras

## Exercício 5

Considere apenas ordens com status entregue e sellers de SP. Encontre os top 5 produtos com maior faturamento (retorne o product_id e o faturamento total). Exiba um DataFrame e também um gráfico de barras

## Exercício 6

Considere apenas ordens com status entregue. Crie uma query SQL que calcule o faturamento mensal da olist em 2017. Exiba o DataFrame e um gráfico de linhas.

Dicas:
- Utilize `date_part` no SQL para extrair mês e ano https://www.postgresql.org/docs/13/functions-datetime.html
- px.line https://plotly.com/python/line-charts/

## Exercício 7

Considere apenas ordens com status entregue. Crie uma query SQL que calcule o faturamento mensal da olist em 2017 por UF do seller. Exiba o DataFrame (contento colunas de mês, UF e faturamento), além de um gráfico de linhas

Dicas:
- Utilize date_part no SQL para extrair mês e ano https://www.postgresql.org/docs/13/functions-datetime.html
- px.line https://plotly.com/python/line-charts/

## Exercício 8

Vamos exibir onde estão localizados os sellers da olist?!

Cruze as tabelas de seller e geolocation, e exiba um mapa de calor contendo aproximadamente 3% da base.